In [1]:
import datetime
import pandas as pd
import mplfinance as mpf
import matplotlib.pyplot as plt

from lutils.fin.data_loader import load

In [2]:
# %matplotlib inline
# %matplotlib notebook
%matplotlib qt

In [3]:
# symbol1 = 'Z:/tq_data/ticks/SHFE.al2203C22800.h5'
# symbol2 = 'Z:/tq_data/ticks/SHFE.al2204C22800.h5'
symbol1 = 'Z:/tq_data/ticks/SHFE.al2306C18700.h5'
symbol2 = 'Z:/tq_data/ticks/SHFE.al2307C18700.h5'

In [4]:
def load_data(path):
    df = pd.read_hdf(path) # SHFE.al2206.h5
    cs = []
    for c in df.columns.values:
        cs.append(c.rsplit('.', 1)[-1])
    columns = dict(zip(df.columns.values, cs))
    df = df.rename(columns=columns)
    df['tradetime'] = pd.to_datetime(df.datetime, format='%Y-%m-%d %H:%M:%S.%f')
    _df = df[(df.tradetime.dt.hour >= 8) | (df.tradetime.dt.hour <= 1)]
    
    tradetime = df.loc[((df.tradetime.dt.dayofweek >= 4) & (df.tradetime.dt.hour >= 20)) | (df.tradetime.dt.dayofweek > 4)].tradetime + datetime.timedelta(days=2)
    _df.loc[tradetime.index, 'tradetime'] = tradetime.values
    
    _df.index = _df.tradetime
    resample_ohlc = _df['last_price'].resample('1Min', closed='left', label='right').ohlc(_method='ohlc')
    
    return resample_ohlc

In [5]:
df1 = load_data(symbol1)

D:\Software\Python\Python37\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [6]:
df1.tail() # [['tradetime', 'last_price']]

,open,high,low,close
tradetime,,,,
2023-04-28 14:56:00,167.0,167.0,167.0,167.0
2023-04-28 14:57:00,167.0,167.0,167.0,167.0
2023-04-28 14:58:00,167.0,167.0,167.0,167.0
2023-04-28 14:59:00,167.0,167.0,156.0,156.0
2023-04-28 15:00:00,156.0,156.0,156.0,156.0


In [7]:
df2 = load_data(symbol2)

D:\Software\Python\Python37\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [8]:
df2.tail() # [['tradetime', 'last_price']]

,open,high,low,close
tradetime,,,,
2023-04-28 14:56:00,273.0,273.0,273.0,273.0
2023-04-28 14:57:00,273.0,273.0,273.0,273.0
2023-04-28 14:58:00,273.0,273.0,273.0,273.0
2023-04-28 14:59:00,273.0,273.0,273.0,273.0
2023-04-28 15:00:00,273.0,273.0,273.0,273.0


In [9]:
# df = pd.merge_asof(df1, df2, on='tradetime')

In [15]:
df = df2.open.sub(df1.open)

In [21]:
df

tradetime
2023-01-04 18:38:00      NaN
2023-01-04 18:39:00      NaN
2023-01-04 18:40:00      NaN
2023-01-04 18:41:00      NaN
2023-01-04 18:42:00      NaN
                       ...  
2023-04-28 14:56:00    106.0
2023-04-28 14:57:00    106.0
2023-04-28 14:58:00    106.0
2023-04-28 14:59:00    106.0
2023-04-28 15:00:00    117.0
Freq: T, Name: open, Length: 163943, dtype: float64

In [12]:
# df.index = df.tradetime

In [13]:
# df.tail()[['last_price_x', 'last_price_y']]

In [21]:
subdf = df.last_price_y - df.last_price_x

In [22]:
subdf

tradetime
2023-01-04 18:37:13.100000      NaN
2023-01-04 18:37:14.500000      NaN
2023-01-04 21:06:04.500000      NaN
2023-01-04 21:06:05.000000      NaN
2023-01-04 21:15:30.000000      NaN
                              ...  
2023-04-28 14:59:16.000000    117.0
2023-04-28 14:59:18.500000    117.0
2023-04-28 14:59:32.500000    117.0
2023-04-28 14:59:59.000001    117.0
2023-04-28 14:59:59.999500    117.0
Length: 287451, dtype: float64

In [23]:
# subdf = subdf.rename('diff_price')

In [24]:
subdf

tradetime
2023-01-04 18:37:13.100000      NaN
2023-01-04 18:37:14.500000      NaN
2023-01-04 21:06:04.500000      NaN
2023-01-04 21:06:05.000000      NaN
2023-01-04 21:15:30.000000      NaN
                              ...  
2023-04-28 14:59:16.000000    117.0
2023-04-28 14:59:18.500000    117.0
2023-04-28 14:59:32.500000    117.0
2023-04-28 14:59:59.000001    117.0
2023-04-28 14:59:59.999500    117.0
Length: 287451, dtype: float64

In [28]:
resample_ohlc = subdf.resample('1Min', closed='left', label='right').ohlc(_method='ohlc').dropna()

In [29]:
resample_ohlc

,open,high,low,close
tradetime,,,,
2023-02-14 15:00:00,124.0,124.0,124.0,124.0
2023-02-15 15:00:00,-19.0,-19.0,-19.0,-19.0
2023-02-16 15:00:00,73.0,73.0,73.0,73.0
2023-02-17 15:00:00,19.0,19.0,19.0,19.0
2023-02-20 15:00:00,4.0,4.0,4.0,4.0
...,...,...,...,...
2023-04-28 14:56:00,106.0,106.0,106.0,106.0
2023-04-28 14:57:00,106.0,106.0,106.0,106.0
2023-04-28 14:58:00,106.0,106.0,106.0,106.0


In [22]:
dfDay = df.resample('24H', closed='left', label='right', offset='21H').agg({
#         'Contract': 'first',
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
#         'Volume': 'sum',
#         'OpenInterest': 'sum'
    }).dropna()

In [23]:
dfDay.tail(10)

,open,high,low,close
tradetime,,,,
2023-04-17 21:00:00,136.0,136.0,136.0,136.0
2023-04-18 21:00:00,100.0,134.0,53.0,68.0
2023-04-19 21:00:00,110.0,145.0,76.0,108.0
2023-04-20 21:00:00,84.0,118.0,-16.0,75.0
2023-04-21 21:00:00,92.0,285.0,32.0,119.0
2023-04-24 21:00:00,107.0,163.0,80.0,120.0
2023-04-25 21:00:00,99.0,137.0,57.0,93.0
2023-04-26 21:00:00,117.0,145.0,93.0,113.0
2023-04-27 21:00:00,125.0,142.0,101.0,134.0


In [32]:
mpf.plot(dfDay, type='candle', volume=False, mav=(3,6,9), figratio=(3,1), style='yahoo', title='df', datetime_format='%Y-%m-%d')